# Midterm 1

## FINM 36700 - 2023

### UChicago Financial Mathematics

* Mark Hendricks
* hendricks@uchicago.edu

# Instructions

## Please note the following:

Points
* The exam is 100 points.
* You have 120 minutes to complete the exam.
* For every minute late you submit the exam, you will lose one point.
Final Exam

Submission
* You will upload your solution to the `Midterm 1` assignment on Canvas, where you downloaded this. (Be sure to **submit** on Canvas, not just **save** on Canvas.
* Your submission should be readable, (the graders can understand your answers,) and it should **include all code used in your analysis in a file format that the code can be executed.** 

Rules
* The exam is open-material, closed-communication.
* You do not need to cite material from the course github repo--you are welcome to use the code posted there without citation.

Advice
* If you find any question to be unclear, state your interpretation and proceed. We will only answer questions of interpretation if there is a typo, error, etc.
* The exam will be graded for partial credit.

## Data

**All data files are found in the class github repo, in the `data` folder.**

This exam makes use of the following data files:
* `midterm_data_1.xlsx`

This file has sheets for...
* `info` - names of each stock ticker
* `excess returns` - weekly excess returns on several stocks
* `SPY` - weekly excess returns on SPY

Note the data is **weekly** so any annualizations should use `52` weeks in a year.

#### If useful
here is code to load in the data.

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import kurtosis, skew
from scipy.stats import norm
import seaborn as sns
import statsmodels.api as sm
from statsmodels.regression.rolling import RollingOLS
import warnings
warnings.filterwarnings("ignore")
from sympy import Matrix, init_printing, matrix2numpy, diag
from scipy.stats import kurtosis

%matplotlib inline

import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (16,6)

In [2]:
FILEIN = '../data/midterm_1_data.xlsx'
sheet_exrets = 'excess returns'
sheet_spy = 'spy'

retsx = pd.read_excel(FILEIN, sheet_name=sheet_exrets).set_index('date')
spy = pd.read_excel(FILEIN, sheet_name=sheet_spy).set_index('date')

## Scoring

| Problem | Points |
|---------|--------|
| 1       | 20     |
| 2       | 35     |
| 3       | 30     |
| 4       | 15     |

### Each numbered question is worth 5 points.

### Notation
(Hidden LaTeX commands)

$$\newcommand{\mux}{\tilde{\boldsymbol{\mu}}}$$
$$\newcommand{\wtan}{\boldsymbol{\text{w}}^{\text{tan}}}$$
$$\newcommand{\wtarg}{\boldsymbol{\text{w}}^{\text{port}}}$$
$$\newcommand{\mutarg}{\tilde{\boldsymbol{\mu}}^{\text{port}}}$$
$$\newcommand{\wEW}{\boldsymbol{\text{w}}^{\text{EW}}}$$
$$\newcommand{\wRP}{\boldsymbol{\text{w}}^{\text{RP}}}$$
$$\newcommand{\wREG}{\boldsymbol{\text{w}}^{\text{REG}}}$$

# 1. Short Answer

### No Data Needed

These problem does not require any data file. Rather, analyze the situation conceptually, based on the information below. 

## 1

In what sense was ProShares `HDG` successful in hedging the `HFRI`, and in what sense was it unsuccessful in tracking the `HFRI`?

It was succesfull in that HDG's correlation with ML Factor's model, which itself tracks the HFRI, was very high. Approximately 99%. However, some of the features of the HFRI were not necessarily appropriately captured. The skewness and kurtosis were not matched as closely.

## 2

We discussed multiple ways of calculating Value-at-Risk (VaR). What are the tradeoffs between using the normal distribution formula versus a directly empirical approach?

The main tradeoff is imposing a statistical model that does not necessarily fit the data in order to gain statistical power. When using the empirical approach, one cosiders only very few (potentially one) observation(s). However, when using the normal distribution formula, we can use the volatility of the dataset as a whole. The empirical approach is in a sense "overfitting" bad events in the data. Using the normal distribution sacrifices precision to gain statistical power. In general, the normal distribution approach seems to perform better at predicting VAR threshold despite including some bias in the process while imposing a potentially inaccurate model onto the data.

## 3

Did we find that **TIPS** have been useful in expanding the mean-variance frontier in the past? Did we conclude they might be useful in the future? Explain.

There is more diversification between TIPS and bonds than between SPY and many other equity buckets Harvard has. As an additional security, TIPS was useful in expanding the mean-variance frontier. They can be used in the future.

## 4.

What aspect of the classic mean-variance optimization approach leads to extreme answers? How did regularization help with this issue?

Because of typically high condition numbers in the covariance matrix, or high sensitivity of its inverse, small variations on expected returns could affect significantly the weights on specific assets. The classic mean-variance optimization approach assumes observed mean returns as certain and concrete values, even though in reality they are hard to estimate extremely accurately. Because of this, the classic mean-variance optimization can take very large positions on specific assets, when in reality potential gains from such positions could be highly uncertain. Regularization prevents precisely such large weights in the final allocation/weights, by penalizing in some form large coefficients. It helps the optimization take observed data "less seriously", or consider the fact that it may not perfectly predict future performance.

***

# 2. Allocation

Consider a mean-variance optimization of **excess** returns provided in `midterm_1_data.xlsx.`

## 1. 

Report the following **annualized** statistics:
* mean
* volatility
* Sharpe ratio

Which assets have the highest / lowest Sharpe ratios?

In [3]:
def summary_statistics_annualized(returns, annual_factor = 52):
    """This functions returns the summary statistics for the input total/excess returns passed
    into the function"""
    
    summary_statistics = pd.DataFrame(index=returns.columns)
    summary_statistics['Mean'] = returns.mean() * annual_factor
    summary_statistics['Vol'] = returns.std() * np.sqrt(annual_factor)
    summary_statistics['Sharpe'] = (returns.mean() / returns.std()) * np.sqrt(annual_factor)
    summary_statistics['Min'] = returns.min()
    summary_statistics['Max'] = returns.max()
    summary_statistics['Skewness'] = returns.skew()
    summary_statistics['Excess Kurtosis'] = returns.kurtosis()
    summary_statistics['VaR (0.05)'] = returns.quantile(.05, axis = 0)
    summary_statistics['CVaR (0.05)'] = returns[returns <= returns.quantile(.05, axis = 0)].mean()
    
    return summary_statistics


stats=summary_statistics_annualized(retsx)
stats

,Mean,Vol,Sharpe,Min,Max,Skewness,Excess Kurtosis,VaR (0.05),CVaR (0.05)
AAPL,0.319421,0.283883,1.125183,-0.190566,0.143562,-0.334342,2.672198,-0.052313,-0.085612
MSFT,0.288087,0.240206,1.199334,-0.150492,0.104231,-0.359175,1.737189,-0.049366,-0.071559
AMZN,0.239457,0.310389,0.771474,-0.151901,0.156111,-0.210630,1.746315,-0.061868,-0.096065
NVDA,0.650658,0.468096,1.390011,-0.210199,0.332580,0.425676,2.244417,-0.083805,-0.119446
GOOGL,0.193328,0.274217,0.705020,-0.135524,0.149258,0.041986,1.143573,-0.055729,-0.078408
TSLA,0.569728,0.607026,0.938556,-0.284957,0.334897,0.441455,1.527376,-0.122519,-0.155313
XOM,0.124196,0.311613,0.398557,-0.175338,0.184173,0.097936,3.129459,-0.061685,-0.097340


**The data is weekly! I am assuming/using 52 weeks in a year (x52) to annualize the values**

Highest Sharpe: NVDA

Lowest Sharpe: XOM

## 2.

Report the weights of the tangency portfolio.

Also report the Sharpe ratio achieved by the tangency portfolio over this sample.

In [4]:
def calculate_statistics(df):
    res={}
    res_i={}
    for i in df.columns:
        if df[i].dtype=='<M8[ns]':
            pass
        else:
            res_i.update({'mean':np.mean(df[i])*52})
            res_i.update({'volatility':np.std(df[i])*(52**(1/2))})
            res_i.update({'sharpe':res_i['mean']/res_i['volatility']})
            res.update({i:res_i})
            res_i={}
    return res

def calculate_statistics_array(data):
    res_i={}
    res_i.update({'mean':np.mean(data)*52})
    res_i.update({'volatility':np.std(data)*(52**(1/2))})
    res_i.update({'sharpe':res_i['mean']/res_i['volatility']})
    return res_i


def correlation_matrix(df):
    return df.corr()

#Make sure the first column is dates
def tangency_portfolio(df):
    stats=calculate_statistics(df)
    assets=len(df.columns)
    mdf=Matrix(df.cov())
    vect1=Matrix([1]*assets)
    mean=[]
    for i in stats:
        mean.append(stats[i]['mean'])
    vectmean=Matrix(mean)
    sigma_inv=mdf.inv()
    wt=(1/((vect1.T@sigma_inv@vectmean)[0,0]))*(sigma_inv@vectmean)

    tickers=[]
    for i in stats:
        tickers.append(i)
    tan_port=pd.DataFrame()
    tan_port['tickers']=tickers
    tan_port['Tangent Weights']=0
    for i in range(len(tan_port)):
        tan_port.loc[i,'Tangent Weights']=wt[i]
    
    tan_port.set_index('tickers', inplace=True,drop=True)

    return tan_port

wt=tangency_portfolio(retsx)
wt

,Tangent Weights
tickers,
AAPL,0.322604866030519
MSFT,0.787495885834573
AMZN,-0.228606530386357
NVDA,0.495995507989400
GOOGL,-0.502721011619707
TSLA,0.105974674197866
XOM,0.0192566079537055


In [5]:
stats_tangency=calculate_statistics_array(tangency_portfolio(retsx)['Tangent Weights']@retsx.T)
print(stats_tangency)
print()
print("Tangency portfolio's Sharpe: ", stats_tangency['sharpe'])

{'mean': 0.5634741093289564, 'volatility': 0.35789342042717837, 'sharpe': 1.5744187435924324}

Tangency portfolio's Sharpe:  1.5744187435924324


## 3.

* What weight is given to the asset with the lowest Sharpe ratio?
* What Sharpe ratio does the lowest (most negative) weight asset have?

Explain. Support your answer with evidence.

In [6]:
twt=wt.copy()
twt = twt.merge(stats[['Sharpe']], left_index=True, right_index=True)
twt

,Tangent Weights,Sharpe
AAPL,0.322604866030519,1.125183
MSFT,0.787495885834573,1.199334
AMZN,-0.228606530386357,0.771474
NVDA,0.495995507989400,1.390011
GOOGL,-0.502721011619707,0.705020
TSLA,0.105974674197866,0.938556
XOM,0.0192566079537055,0.398557


The asset with the lowest sharpe ratio (XOM) gets a weight of 0.019.

The asset with the lowest weight (GOOGL) has a sharpe of ~0.705.

The weights are highly dependent on the correlation/covariance among available assets, more than on the individual sharpes or individual performances themselves. This effect is accentuated the more assets we consider in the optimization.

## 4.

Let's examine the out-of-sample performance.

Calculate and report the following three allocations using only data through the end of 2022:
* tangency portfolio
* equally weighted portfolio
* a regularized approach, with a new formula shown below

where
$$\wEW_i = \frac{1}{n}$$

$$\wREG \sim \widehat{\Sigma}^{-1}\mux$$

$$\widehat{\Sigma} = \frac{\Sigma + \boldsymbol{2}\,\Sigma_D}{\boldsymbol{3}}$$
where $\Sigma_D$ denotes a *diagonal* matrix of the security variances, with zeros in the off-diagonals.

In [7]:
def tangency_portfolio_rfr(asset_return,cov_matrix, cov_diagnolize = False):
    """ 
        Returns the tangency portfolio weights in a (1 x n) vector
        Inputs: 
            asset_return - return for each asset (n x 1) Vector
            cov_matrix = nxn covariance matrix for the assets
    """
    if cov_diagnolize:
        asset_cov = np.diag(np.diag(cov_matrix))
    else:
        asset_cov = np.array(cov_matrix)
    inverted_cov= np.linalg.inv(asset_cov)
    one_vector = np.ones(len(cov_matrix.index))
    
    den = (one_vector @ inverted_cov) @ (asset_return)
    num =  inverted_cov @ asset_return
    return (1/den) * num


Tangency Portfolio:

In [8]:
retsx_2022 = retsx.loc[:'2022-12-31'].copy()
TangencyPort_df_2022 = pd.DataFrame(tangency_portfolio_rfr(retsx_2022.mean(), retsx_2022.cov()),columns= ["Tangency Portfolio Weight"],index=retsx.columns)
TangencyPort_df_2022


,Tangency Portfolio Weight
AAPL,0.310565
MSFT,1.073114
AMZN,-0.259080
NVDA,0.380133
GOOGL,-0.751548
TSLA,0.101559
XOM,0.145257


Equally weighted portfolio:

In [9]:
Equally_weighted = pd.DataFrame(columns= ["EW Portfolio Weight"],index=retsx.columns)
Equally_weighted["EW Portfolio Weight"] = 1/len(retsx.columns)
Equally_weighted

,EW Portfolio Weight
AAPL,0.142857
MSFT,0.142857
AMZN,0.142857
NVDA,0.142857
GOOGL,0.142857
TSLA,0.142857
XOM,0.142857


Regularized approach

In [10]:
cov_matrix = retsx.cov()
cov_diag = np.diag(np.diag(cov_matrix))
cov_matrix_reg = (cov_matrix + 2*cov_diag)/3
cov_matrix_reg_inv = np.linalg.inv(cov_matrix_reg)
mean_2022 = retsx_2022.mean()
reg_weights = cov_matrix_reg_inv @ mean_2022
Reg_weights = pd.DataFrame(columns= ["Reg Portfolio Weight"],index=retsx.columns)
Reg_weights["Reg Portfolio Weight"] = reg_weights
Reg_weights


,Reg Portfolio Weight
AAPL,2.175852
MSFT,2.715273
AMZN,0.418795
NVDA,1.646301
GOOGL,0.141063
TSLA,0.734261
XOM,0.779944


## 5.

Report the out-of-sample (2023) performance of all three portfolios in terms of annualized mean, vol, and Sharpe.

In [11]:
# Tangency Portfolio
tangency_returns_2023 = (TangencyPort_df_2022['Tangency Portfolio Weight'] * retsx.loc['2023']).sum(axis=1)
tangency_stats_2023 = calculate_statistics_array(tangency_returns_2023)

print("Tangency Portfolio Summary Statistics for 2023:\n", tangency_stats_2023)

# Equally Weighted Portfolio
ew_returns_2023 = (Equally_weighted['EW Portfolio Weight'] * retsx.loc['2023']).sum(axis=1)
ew_stats_2023 = calculate_statistics_array(ew_returns_2023)

print("Equally Weighted Portfolio Summary Statistics for 2023:\n", ew_stats_2023)

# Regularized Portfolio
reg_returns_2023 = (Reg_weights['Reg Portfolio Weight'] * retsx.loc['2023']).sum(axis=1)
reg_stats_2023 = calculate_statistics_array(reg_returns_2023)

print("Regularized Portfolio Summary Statistics for 2023:\n", reg_stats_2023)


Tangency Portfolio Summary Statistics for 2023:
 {'mean': 1.204709053182053, 'volatility': 0.43572077841290124, 'sharpe': 2.7648648236840265}
Equally Weighted Portfolio Summary Statistics for 2023:
 {'mean': 0.9551326432690281, 'volatility': 0.24250313271806, 'sharpe': 3.9386404314186256}
Regularized Portfolio Summary Statistics for 2023:
 {'mean': 8.585486903792894, 'volatility': 2.0746505224419893, 'sharpe': 4.138281031393787}


## 6.

Imagine just for this problem that this data is for **total** returns, not excess returns.

Report the weights of the global-minimum-variance portfolio.

In [12]:
def gmv_portfolio(cov_matrix):
    """ 
        Returns the Global Minimum Variance portfolio weights in a (1 x n) vector
        Inputs: 
            asset_return - return for each asset (n x 1) Vector
            cov_matrix = nxn covariance matrix for the assets
    """
    asset_cov = np.array(cov_matrix)
    inverted_cov= np.linalg.inv(asset_cov)
    one_vector = np.ones(len(cov_matrix.index))
    
    den = (one_vector @ inverted_cov) @ (one_vector)
    num =  inverted_cov @ one_vector
    return (1/den) * num

gmv_weights = gmv_portfolio(retsx.cov())
gmv_df = pd.DataFrame(gmv_weights, columns=['GMV Portfolio Weight'], index=retsx.columns)
gmv_df


,GMV Portfolio Weight
AAPL,0.206231
MSFT,0.491250
AMZN,0.160866
NVDA,-0.119168
GOOGL,0.011378
TSLA,-0.046927
XOM,0.296369


## 7.

To target a mean return of 0.005%, would you be long or short this global minimum variance portfolio?

In [13]:
gmv_returns = (gmv_df['GMV Portfolio Weight'] * retsx).sum(axis=1)
gmv_stats = calculate_statistics_array(gmv_returns)

print("GMV Portfolio Summary Statistics for 2023:\n", gmv_stats)


GMV Portfolio Summary Statistics for 2023:
 {'mean': 0.180652235492206, 'volatility': 0.20264617069916296, 'sharpe': 0.8914663172214198}


Assuming there is a rf asset, this portfolio would not be involved, as the optimal option would be a combination of the rf asset and the tangency portolio.

However, assuming that there is no rf asset and that the 0.005% return is not annualized:

Also, using the entire dataset

In [14]:
gmv_stats['mean']/52

0.003474081451773192

In [15]:
stats_tangency['mean']/52

0.010836040564018391

You would be long! The mean is in-between the minimim variance and the tangency portfolio so no need to short.

***

# 3. Performance

## 1. 

Report the following performance metrics of excess returns for Tesla (`TSLA`).
* skewness
* kurtosis

You are not annualizing any of these stats.

What do these metrics indicate about the nature of the returns?

In [16]:
def performance_summary(return_data, annualization = 52):
    """ 
        Returns the Performance Stats for given set of returns
        Inputs: 
            return_data - DataFrame with Date index and Monthly Returns for different assets/strategies.
        Output:
            summary_stats - DataFrame with annualized mean return, vol, sharpe ratio. Skewness, Excess Kurtosis, Var (0.5) and
                            CVaR (0.5) and drawdown based on monthly returns. 
    """
    summary_stats = return_data.mean().to_frame('Mean').apply(lambda x: x*annualization)
    summary_stats['Volatility'] = return_data.std().apply(lambda x: x*np.sqrt(annualization))
    summary_stats['Sharpe Ratio'] = summary_stats['Mean']/summary_stats['Volatility']

    summary_stats['Skewness'] = return_data.skew()
    summary_stats['Excess Kurtosis'] = return_data.kurtosis()
    summary_stats['VaR (0.05)'] = return_data.quantile(.05, axis = 0)
    summary_stats['CVaR (0.05)'] = return_data[return_data <= return_data.quantile(.05, axis = 0)].mean()
    
    wealth_index = 1000*(1+return_data).cumprod()
    previous_peaks = wealth_index.cummax()
    drawdowns = (wealth_index - previous_peaks)/previous_peaks

    summary_stats['Max Drawdown'] = drawdowns.min()
    summary_stats['Peak'] = [previous_peaks[col][:drawdowns[col].idxmin()].idxmax() for col in previous_peaks.columns]
    summary_stats['Bottom'] = drawdowns.idxmin()
    
    recovery_date = []
    for col in wealth_index.columns:
        prev_max = previous_peaks[col][:drawdowns[col].idxmin()].max()
        recovery_wealth = pd.DataFrame([wealth_index[col][drawdowns[col].idxmin():]]).T
        recovery_date.append(recovery_wealth[recovery_wealth[col] >= prev_max].index.min())
    summary_stats['Recovery'] = recovery_date
    
    return summary_stats

perf_metrics=performance_summary(retsx).loc['TSLA']


print('Skewness: ', perf_metrics['Skewness'])
print('Kurtosis: ', perf_metrics['Excess Kurtosis'])



Skewness:  0.44145458268186166
Kurtosis:  1.5273762473269983


Positive skewness in a dataset indicates that the data is skewed or biased towards the right side of the distribution. In other words, the tail on the right side of the distribution is longer or fatter than the left side. 

Excess kurtosis is a measure of the shape of a probability distribution, specifically focusing on the tails of the distribution. Positive excess kurtosis indicates that the distribution has fatter or heavier tails compared to a normal distribution.

## 2. 

Report the maximum drawdown for `TSLA` over the sample.
* Ignore that your data is in excess returns rather than total returns.
* Simply proceed with the excess return data for this calculation.

In [17]:
print('Max Drawdown: ',perf_metrics['Max Drawdown'])
print('Peak: ',perf_metrics['Peak'])
print('Bottom: ',perf_metrics['Bottom'])
print('Recovery: ',perf_metrics['Recovery'])

Max Drawdown:  -0.6821852296331565
Peak:  2021-11-05 00:00:00
Bottom:  2023-01-06 00:00:00
Recovery:  NaT


## 3.

For `TSLA`, calculate the following metrics, relative to `SPY`:
* market beta
* alpha
* sortino ratio

Annualize alpha and sortino ratio.

In [18]:
def regression_based_performance(factor,fund_ret,rf,constant = True):
    """ 
        Returns the Regression based performance Stats for given set of returns and factors
        Inputs:
            factor - Dataframe containing monthly returns of the regressors
            fund_ret - Dataframe containing monthly excess returns of the regressand fund
            rf - Monthly risk free rate of return
        Output:
            summary_stats - (Beta of regression, treynor ratio, information ratio, alpha). 
    """
    if constant:
        X = sm.tools.add_constant(factor)
    else:
        X = factor
    y=fund_ret
    model = sm.OLS(y,X,missing='drop').fit()
    
    if constant:
        beta = model.params[1:]
        alpha = round(float(model.params['const']),6)
        
    else:
        beta = model.params
    treynor_ratio = ((fund_ret.values-rf.values).mean()*12)/beta[0]
    tracking_error = (model.resid.std()*np.sqrt(12))
    if constant:        
        information_ratio = model.params[0]*12/tracking_error
    r_squared = model.rsquared
    if constant:
        return (beta,treynor_ratio,information_ratio,alpha,r_squared,tracking_error,alpha)
    else:
        return (beta,treynor_ratio,r_squared,tracking_error,alpha)

In [19]:


# calculate market beta, alpha and sortino ratio

rf = pd.Series(np.zeros(len(retsx)), index=retsx.index) # create a series of zeros with the same length as retsx

res = regression_based_performance(spy, retsx['TSLA'], rf=rf)
alpha=res[-1]
beta=res[0]

alpha_annual = (1 + alpha) ** 52 - 1

# print results
print("Market Beta for TSLA: ", beta)
print("Alpha for TSLA: ", alpha_annual)

def sortino_ratio(returns, rf=0, target=0):
    """
    Calculate Sortino Ratio for a given set of returns
    """
    downside_returns = returns.copy()
    downside_returns[returns >= target] = 0
    downside_deviation = np.sqrt((downside_returns ** 2).sum() / len(returns)) * np.sqrt(12)
    sortino_ratio = (returns.mean() - rf) / downside_deviation
    return sortino_ratio

sortino = sortino_ratio(retsx['TSLA'] - rf, target=0)
print("Sortino Ratio for TSLA: ", sortino*np.sqrt(52))


Market Beta for TSLA:  SPY    1.776825
dtype: float64
Alpha for TSLA:  0.36142897135532337
Sortino Ratio for TSLA:  0.4512712557740471


## 4.

Continuing with `TSLA`, calculate the full-sample, 5th-percentile CVaR.
* Use the `normal` formula, assuming mean returns are zero.
* Use the full-sample volatility.

Use the entire sample to calculate a single CVaR number. 

In [20]:
import scipy.stats as stats

# calculate z-score for 5th percentile
z = stats.norm.ppf(0.05)

# calculate mean and standard deviation of excess returns
mean = retsx['TSLA'].mean()
std = retsx['TSLA'].std()

# calculate CVaR using normal formula
cvar = - norm.pdf(z) / 0.05 * std

print("Full-sample 5th percentile CVaR: ", retsx['TSLA'][retsx['TSLA'] <= cvar].mean())


Full-sample 5th percentile CVaR:  -0.2385757255683332


## 5.

Now calculate the 5th-percentile, one-period ahead, **VaR** for `TSLA`.

Here, calculate the running series of VaR estimates.

Again, 
* use the normal formula, with mean zero.

But now, use the rolling volatility, based on 
* rolling window or $m=52$ weeks.

Report the final 5 values of your calculated VaR series.

In [21]:
window = 52
volatility = retsx['TSLA'].rolling(window).std() * np.sqrt(window)
var = -volatility * stats.norm.ppf(0.05)

print(var.tail(5))


date
2023-06-16    1.133727
2023-06-23    1.117533
2023-06-30    1.104626
2023-07-07    1.092999
2023-07-14    1.088110
Name: TSLA, dtype: float64


## 6. 

Calculate the out-of-sample **hit ratio** for your VaR series reported in your previous answer.

***

# 4. Hedging

## 1. 

Consider the following scenario: you are holding a \$100 million long position in `NVDA`. You wish to hedge the position using some combination of 
* `AAPL`
* `AMZN`
* `GOOGL`
* `MSFT`

Report the positions you would hold of those 4 securities for an optimal hedge.

Note:
* In the regression estimation, include an intercept.
* Use the full-sample regression. No need to worry about in-sample versus out-of-sample.

The positions of each asset would be the negative of the betas of the regression

In [22]:
import statsmodels.api as sm

# create a dataframe with the selected stocks
df = pd.concat([retsx['AAPL'], retsx['AMZN'], retsx['GOOGL'], retsx['MSFT']], axis=1)

# add a constant column to the dataframe
df = sm.add_constant(df)

# run the regression
model = sm.OLS(retsx['NVDA'], df).fit()

# print the betas
print(model.params)

const    0.005264
AAPL     0.341686
AMZN     0.417260
GOOGL   -0.007848
MSFT     0.587897
dtype: float64


The positions in million for a 100 million NVDA long position below:

In [23]:
model.params[1:]*100

AAPL     34.168649
AMZN     41.725986
GOOGL    -0.784795
MSFT     58.789673
dtype: float64

## 2.

How well does the hedge do? Cite a regression statistic to support your answer.

Also estimate the volatility of the basis, (epsilon.)

In [24]:
residuals = retsx['NVDA'] - model.predict(df)
residual_volatility = residuals.std()
print("Volatility of residuals: ", residual_volatility)

Volatility of residuals:  0.047782087688036136


In [25]:
print("R-squared of the regression: ", model.rsquared)

R-squared of the regression:  0.45816823611553836


An R² value of 0.46 indicates that approximately 46% of the variability in the dependent variable is explained/captured by the assets we are using to hedge.

The assets used are not capturing most of the variability seen in NVDA.

## 3.

Report the annualized intercept. By including this intercept, what are you assuming about the nature of the returns of `NVDA` as well as the returns of the hedging instruments?

In [26]:
intercept = model.params['const']
intercept_annual = (1 + intercept) ** 52 - 1
print("Annualized intercept: ", intercept_annual)


Annualized intercept:  0.31394506967436286


We are assuming that the differences in levels of the returns observed are not necessarily likely to persist in the future, so we want the betas to focus on the variability, as opposed to the difference in the levels themselves.